In [1]:
# !pip install torch==2.2.2+cu121 torchvision==0.17.2+cu121 torchaudio==2.2.2+cu121 -f https://pypi.tuna.tsinghua.edu.cn/simple -f https://download.pytorch.org/whl/torch_stable.html
# !pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# TASK 1

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import logging
import random
from functools import partial
import sklearn
from sklearn.model_selection import StratifiedKFold
import datasets
import transformers
import torch
import copy
from torch import nn
import gc
import torch.nn.functional as F
from tqdm.auto import tqdm, trange
from scipy.special import softmax, expit
logger = logging.getLogger(__name__)

D:\MiniConda\envs\pythonProject18\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def set_seeds(self, seed=2023): 
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [3]:
# Source: https://huggingface.co/l3cube-pune/assamese-bert
params = dict(learning_rate=1.0e-5, seed=2023, data_seed=2023, n_splits=10,
              model_id='models--l3cube-pune--bengali-bert', tensorboard_dir='tensorboard', label_col='task_1',
              patience=5, metric='f1', metric_mode='max', train_batch_size=32, eval_batch_size=32,
              train_max_length=512, eval_max_length=512, learning_rate_scheduling='cosine_annealing',
              gradient_accumulation_steps=1, warmup_proportion=50, num_train_epochs=100, max_seq=512,
              fp16=False, logging_steps=None, model_dir='/model/', password='Bangla')

y_map = {0.0: 'Negative', 1.0: 'Neutral', 2.0: 'Positive'}

In [4]:
set_seeds(params['seed'])

# Data Preparation

In [5]:
import getpass
import zipfile
from pathlib import Path
def prepare_data(path, password, train='Bangla_train_data.csv', test='bengali_test_data_wo_label.csv'):
    zfs = [zipfile.ZipFile(path / f.replace('.csv', '.zip')) for f in [train, test]]
    return [pd.read_csv(zfs[i].open(f, 'r', password)) for i, f in enumerate([train, test])]

def split(train_df, y, n_splits=5, data_seed=2023, shuffle=True):
    skf = StratifiedKFold(n_splits=n_splits, random_state=data_seed, shuffle=shuffle)
    train_df['split'] = -1
    for i, (train_index, test_index) in enumerate(skf.split(train_df.index, y)):
        train_df.loc[train_df.index.isin(test_index), 'split'] = i
    return train_df        

path = Path('Bangla')
password =  bytes(getpass.getpass(f"Input file password") if 'password' not in params else params['password'], 'utf-8')
train_df, test_df = prepare_data(path, password=password)
train_df = split(train_df, train_df['Sentiment'], n_splits=params['n_splits'], data_seed=params['data_seed'])
train_df['task_1'] = train_df['Sentiment'].map({v: k for k, v in y_map.items()})
print(len(train_df), len(test_df))

2693 1821


In [6]:
def prepare_feature(df, y_col, tokenizer, max_length):
    texts = df['OCR'].astype(str).tolist()
    features = tokenizer(texts, padding=False, truncation=True, return_tensors=None, max_length=max_length).data
    if y_col in df.columns:
        features['labels'] = df[y_col].tolist()
    return features

In [7]:
train_df.head(10)

,Ids,Sentiment,Sarcasm,Vulgar,Abuse,Target,OCR,split,task_1
0,image_ben_3635.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,I don't need any fucking GIRL বাঁরা কি সাইলেন...,8,1.0
1,image_ben_1586.png,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"After a little problem in relationship Boys, G...",1,0.0
2,image_ben_4040.jpg,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"মা জানো, পাশের বাড়ির চুলবুলি আমার থেকে বেশি অ...",1,0.0
3,image_ben_5074.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,Gf কে ফিঙ্গারং করে দেওয়ার পর যখন সে চরমসুখ অনু...,0,1.0
4,image_ben_7351.jpg,Negative,Sarcastic,Vulgar,Abusive,Gender,টিপতে টিপতে যখন ব্যাথ্যা অনুভব হয়,5,0.0
5,image_ben_3945.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,দাদু হবে নাকি..? টমি ও মিচেল যথাক্রমে 20 ও 25 ...,6,1.0
6,image_ben_4715.jpg,Negative,Sarcastic,Vulgar,Abusive,Gender,Dildo:- I am the best sex-toy for women লে নার...,9,0.0
7,image_ben_4801.jpg,Positive,Sarcastic,Non Vulgar,Abusive,Gender,এক রাউন্ড খেলার পর ছেলেরা পর পর দুই রাউন্ড খেল...,9,2.0
8,image_ben_7196.jpg,Negative,Sarcastic,Non Vulgar,Abusive,Gender,ফেচবুকে ডিপিতে ময়দা মেখেই রাখব সে রিয়ালিটিতে আ...,9,0.0
9,image_ben_4743.jpg,Negative,Sarcastic,Vulgar,Abusive,Gender,মন্দিরে গিয়ে ঠাকুর দেখার সময় যখন ফাঁকা রুমে ...,0,0.0


In [8]:
train_df.loc[train_df['task_1'] == 1.0, 'task_1'] = 0.0
train_df.loc[train_df['task_1'] == 2.0, 'task_1'] = 1.0
print(train_df['task_1'].unique())

[0. 1.]


In [9]:
train_df.head(10)

,Ids,Sentiment,Sarcasm,Vulgar,Abuse,Target,OCR,split,task_1
0,image_ben_3635.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,I don't need any fucking GIRL বাঁরা কি সাইলেন...,8,0.0
1,image_ben_1586.png,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"After a little problem in relationship Boys, G...",1,0.0
2,image_ben_4040.jpg,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"মা জানো, পাশের বাড়ির চুলবুলি আমার থেকে বেশি অ...",1,0.0
3,image_ben_5074.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,Gf কে ফিঙ্গারং করে দেওয়ার পর যখন সে চরমসুখ অনু...,0,0.0
4,image_ben_7351.jpg,Negative,Sarcastic,Vulgar,Abusive,Gender,টিপতে টিপতে যখন ব্যাথ্যা অনুভব হয়,5,0.0
5,image_ben_3945.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,দাদু হবে নাকি..? টমি ও মিচেল যথাক্রমে 20 ও 25 ...,6,0.0
6,image_ben_4715.jpg,Negative,Sarcastic,Vulgar,Abusive,Gender,Dildo:- I am the best sex-toy for women লে নার...,9,0.0
7,image_ben_4801.jpg,Positive,Sarcastic,Non Vulgar,Abusive,Gender,এক রাউন্ড খেলার পর ছেলেরা পর পর দুই রাউন্ড খেল...,9,1.0
8,image_ben_7196.jpg,Negative,Sarcastic,Non Vulgar,Abusive,Gender,ফেচবুকে ডিপিতে ময়দা মেখেই রাখব সে রিয়ালিটিতে আ...,9,0.0
9,image_ben_4743.jpg,Negative,Sarcastic,Vulgar,Abusive,Gender,মন্দিরে গিয়ে ঠাকুর দেখার সময় যখন ফাঁকা রুমে ...,0,0.0


# Prepare Transformer Trainer

In [10]:
class Model(nn.Module):
    def __init__(self, transformer):
        super().__init__()
        self.transformer = transformer 
        
    def m_loss(self, logits, labels):
        return F.binary_cross_entropy_with_logits(logits.squeeze(-1), labels)
        
    def forward(self, **kwargs):
        labels = kwargs.pop("labels", None)
        output = self.transformer(**kwargs)
        output.loss = self.m_loss(output.logits, labels) if labels != None else output.loss
        output.logits = F.sigmoid(output.logits)
        return output
    
    def load(self, model_dir):
        self.transformer.from_pretrained(model_dir)
        print(f"Model loaded: {model_dir}")
    
    def save(self, model_dir):
        self.transformer.save_pretrained(model_dir)
    
class PTTrainer(transformers.Trainer):
    def __init__(self, **kwargs): 
        super().__init__(**kwargs)
        self.compute_metrics = self._metrics if self.eval_dataset is not None else None
        self._patience = 0

    def _metrics(self, eval_preds):
        """
        Evaluation metrics
        """
        threshold = 0.5
        thresholds = np.arange(0.01, 1.0, 0.01) 
        scores, labels = eval_preds
        acc, f1, thr = -1, -1, 0
        auc = sklearn.metrics.roc_auc_score(labels, scores)
        for threshold in thresholds:
            predictions = (scores >= threshold).astype(int)
            new_f1 = sklearn.metrics.f1_score(labels, predictions, average='macro')
            if new_f1 > f1:
                acc = sklearn.metrics.accuracy_score(labels, predictions)
                f1 = new_f1
                thr = threshold
        return {'auc': auc, 'accuracy': acc, 'f1': f1, 'thr': thr}
        
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        forward/backward step
        """
        output = model(**inputs)
        return (output.loss, {'logits': output.logits}) if return_outputs else output.loss

    def _save_checkpoint(self, model, trial, metrics=None):
        # Determine the new best metric / best model checkpoint
        if metrics is not None and self.args.metric_for_best_model is not None:
            metric_to_check = self.args.metric_for_best_model
            if not metric_to_check.startswith("eval_"):
                metric_to_check = f"eval_{metric_to_check}"
            metric_value = metrics[metric_to_check]

            operator = np.greater if self.args.greater_is_better else np.less
            if (
                self.state.best_metric is None
                # or self.state.best_model_checkpoint is None
                or operator(metric_value, self.state.best_metric)
            ):
                self.state.best_metric = metric_value
                # self.state.best_model_checkpoint = self.args.output_dir
                model.save(self.args.output_dir)
                print(f'Model Saved: {metric_value}')
                self._patience = 0
            else:
                print(f"No improvments: {metric_value}, patience: {self._patience}, lr: {self.optimizer.param_groups[0]['lr']}")
                self._patience += 1
            # LR annealing
            if params['learning_rate_scheduling'].lower() == 'cosine_annealing':
                if (self._patience - 1) == int(round(params['patience'] / 2)):
                    if not isinstance(self.lr_scheduler, torch.optim.lr_scheduler.CosineAnnealingLR):
                        self.lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=10, eta_min=max(self.args.learning_rate * 0.01, 1.0e-7), verbose=False)
                        print(f'Starting CosineAnnealingLR schedular.')
                        self._patience -= 1
        else:
            model.save(self.args.output_dir)
            print('Model Saved')


class PTDefaultFlowCallback(transformers.DefaultFlowCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        control = super().on_epoch_end(args, state, control, **kwargs)
        # Evaluate
        if args.evaluation_strategy != transformers.IntervalStrategy.EPOCH \
            and state.epoch == args.num_train_epochs:
            control.should_evaluate = state.best_metric is not None
            control.should_save = True
        return control
    
def get_best_thr(history):
    max_i = -1
    for i, record in enumerate(history):
        if f"eval_{params['metric']}" in record:
            if max_i == -1 or record[f"eval_{params['metric']}"] > history[max_i][f"eval_{params['metric']}"]:
                max_i = i
    return history[max_i]['eval_thr'], history[max_i]['eval_f1']

def apply_thr(y, threshold, mapper=None):
    if mapper is None:
        return (y > threshold).astype(int)
    else:
        return [mapper[x] for x in (y > threshold).astype(int)]

def inference(pred_features, threshold=None):
    print(f"Threshold: {threshold}")
    result = trainer.predict(datasets.Dataset.from_dict(dict(pred_features))).predictions
    if threshold is None:
        test_df[params['label_col']] = result[:, 0]
    else:
        test_df[params['label_col']] = apply_thr(result[:, 0], threshold, y_map)
    return test_df[[col for col in test_df.columns if col != 'text']]

In [11]:
transformer = transformers.AutoModelForSequenceClassification.from_pretrained(params['model_id'], num_labels=1)
_tokenizer = transformers.AutoTokenizer.from_pretrained(params['model_id'], model_max_length=params['max_seq'])
callbacks = [PTDefaultFlowCallback(), 
             transformers.EarlyStoppingCallback(early_stopping_patience=params['patience'])]

if isinstance(params['logging_steps'], int):
    save_strategy = transformers.IntervalStrategy.STEPS
else:
    save_strategy = transformers.IntervalStrategy.EPOCH 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at models--l3cube-pune--bengali-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def get_trainer(valid_split=0):
    _model = Model(copy.deepcopy(transformer))
    optimizer = torch.optim.AdamW(_model.parameters(), lr=params['learning_rate'])
    model_dir = f"{params['model_dir']}{valid_split}"
    t_df, v_df = train_df[train_df['split'] != valid_split], train_df[train_df['split'] == valid_split]
    print(f"Train: {len(t_df)}, Valid: {len(v_df)}")
    train_features = prepare_feature(t_df, params['label_col'], _tokenizer, max_length=params['train_max_length'])
    valid_features = prepare_feature(v_df, params['label_col'], _tokenizer, max_length=params['train_max_length'])
    trainer = PTTrainer(model=_model,
        train_dataset=datasets.Dataset.from_dict(dict(train_features)),
        eval_dataset=datasets.Dataset.from_dict(dict(valid_features)) if valid_features is not None else None,
        args=transformers.TrainingArguments(
            label_names=['labels'],
            seed=params['seed'],
            data_seed=params['data_seed'],
            remove_unused_columns=False,
            report_to="tensorboard" if params['tensorboard_dir'] is not None else "none",
            logging_dir=params['tensorboard_dir'],
            per_device_train_batch_size=params['train_batch_size'], 
            per_device_eval_batch_size=params['eval_batch_size'],
            gradient_accumulation_steps=params['gradient_accumulation_steps'],
            warmup_steps=params['warmup_proportion'], 
            # max_steps=20,
            num_train_epochs=params['num_train_epochs'],
            learning_rate=params['learning_rate'], 
            fp16=params['fp16'],
            metric_for_best_model=params['metric'],
            # gradient_checkpointing=fp16,
            # evaluate_during_training=True,
            logging_steps=params['logging_steps'], 
            output_dir=model_dir,
            evaluation_strategy=save_strategy,
            save_strategy=save_strategy,
            logging_strategy=save_strategy,
            save_total_limit=1,
            greater_is_better=params['metric_mode'], 
            load_best_model_at_end=True,
            eval_steps=params['logging_steps'],
            save_steps=params['logging_steps'],
            overwrite_output_dir=True,
            max_grad_norm=1.0,
        ),
        optimizers=[optimizer, transformers.get_constant_schedule(optimizer)],
        callbacks=callbacks,
        data_collator=transformers.DataCollatorWithPadding(_tokenizer),
    )
    return _model, trainer, model_dir

# Train / Inference

In [13]:
pred_features = prepare_feature(test_df, params['label_col'], _tokenizer, max_length=params['eval_max_length'])
subm_df = None
thresholds = []
scores = []
cols = []
for fold in range(params['n_splits']):
    print(f"Fold: {fold + 1} out of {params['n_splits']}")
    col = f'y_{fold}'
    model, trainer, model_dir = get_trainer(fold)
    trainer.train()
    trainer.model.load(model_dir)
    subm = inference(pred_features)
    t, f1 = get_best_thr(trainer.state.log_history)
    thresholds.append(t)
    scores.append(f1)
    if subm_df is None:
        subm_df = subm
    subm_df[col] = subm[params['label_col']]
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    cols.append(col)

Fold: 1 out of 10
Train: 2423, Valid: 270


D:\MiniConda\envs\pythonProject18\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Auc,Accuracy,F1,Thr
1,0.681500,0.661997,0.576125,0.662963,0.398664,0.450000
2,0.655900,0.647622,0.572718,0.662963,0.398664,0.410000
3,0.644100,0.639545,0.612376,0.644444,0.567308,0.380000
4,0.636400,0.634742,0.585303,0.655556,0.575923,0.350000
5,0.627100,0.631180,0.625821,0.692593,0.586004,0.330000
6,0.615400,0.600430,0.701885,0.677778,0.657954,0.380000
7,0.594000,0.605611,0.656332,0.696296,0.656403,0.290000
8,0.566100,0.583585,0.692492,0.725926,0.661218,0.500000
9,0.522900,0.601170,0.682853,0.722222,0.677028,0.630000
10,0.505800,0.598528,0.686875,0.700000,0.661555,0.320000


Model Saved: 0.3986636971046771
No improvments: 0.3986636971046771, patience: 0, lr: 1e-05
Model Saved: 0.5673076923076923
Model Saved: 0.5759233926128591
Model Saved: 0.5860043228464281
Model Saved: 0.6579541317801237
No improvments: 0.6564032528400273, patience: 0, lr: 1e-05
Model Saved: 0.6612181226261531
Model Saved: 0.6770283417598366
No improvments: 0.6615546511088071, patience: 0, lr: 1e-05
No improvments: 0.6612181226261531, patience: 1, lr: 1e-05
No improvments: 0.6681361543125912, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.6497145822522055, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.651070316124329, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/0
Threshold: None


Fold: 2 out of 10
Train: 2423, Valid: 270


...

Model Saved: 0.42944004524886875
No improvments: 0.421505376344086, patience: 0, lr: 1e-05
Model Saved: 0.4472602739726027
Model Saved: 0.49936330688608094
Model Saved: 0.6049034007456785
No improvments: 0.5720624078695797, patience: 0, lr: 1e-05
No improvments: 0.5965175427155341, patience: 1, lr: 1e-05
No improvments: 0.5695274993683146, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.5729542502525617, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.5661290322580645, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/8
Threshold: None


Fold: 10 out of 10
Train: 2424, Valid: 269


D:\MiniConda\envs\pythonProject18\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Auc,Accuracy,F1,Thr
1,0.680600,0.661862,0.514755,0.661710,0.398210,0.450000
2,0.655100,0.648347,0.575225,0.661710,0.398210,0.410000
3,0.645200,0.642207,0.570379,0.661710,0.398210,0.380000
4,0.640400,0.637776,0.597790,0.672862,0.575162,0.360000
5,0.632300,0.628117,0.621558,0.665428,0.584021,0.390000
6,0.606100,0.622462,0.637054,0.646840,0.602325,0.400000
7,0.559500,0.646235,0.622916,0.598513,0.571959,0.310000
8,0.510300,0.637139,0.646500,0.646840,0.608682,0.330000
9,0.480200,0.630796,0.659588,0.665428,0.626343,0.400000
10,0.419400,0.640277,0.665267,0.669145,0.629494,0.230000


Model Saved: 0.3982102908277405
No improvments: 0.3982102908277405, patience: 0, lr: 1e-05
No improvments: 0.3982102908277405, patience: 1, lr: 1e-05
Model Saved: 0.5751615218951902
Model Saved: 0.5840206185567011
Model Saved: 0.6023248937925024
No improvments: 0.571958981612447, patience: 0, lr: 1e-05
Model Saved: 0.6086823367276626
Model Saved: 0.6263427583652303
Model Saved: 0.6294937864648622
Model Saved: 0.6320668693009119
No improvments: 0.6314784884168398, patience: 0, lr: 1e-05
No improvments: 0.6209933075026417, patience: 1, lr: 1e-05
No improvments: 0.6132135604275062, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.6227320371436587, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.6275527864312911, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/9
Threshold: None


In [14]:
thresholds, scores

([0.63, 0.56, 0.38, 0.33, 0.38, 0.3, 0.75, 0.12, 0.41000000000000003, 0.23],
 [0.6770283417598366,
  0.6153626987292723,
  0.631262027438078,
  0.6560102301790282,
  0.6132135604275062,
  0.6315938606847697,
  0.664349376114082,
  0.635567196531792,
  0.6049034007456785,
  0.6320668693009119])

In [15]:
y_map = {0.0: 'Negative', 1.0: 'Positive'}
thr = np.array(thresholds).mean()
print(f"Best threshold: {thr}")
print(f"Best score: {np.array(scores).mean()}")
subm_df['avg'] = subm_df[cols].apply('mean', axis=1)
subm_df[params['label_col']] = apply_thr(subm_df['avg'], thr, y_map)

Best threshold: 0.409
Best score: 0.6361357561910955


In [17]:
subm_df.head(20)

,Ids,OCR,task_1,y_0,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,avg
0,image_ben_1422.png,বৌ-এর প্রেগনেন্সি রিপোর্ট পজিটিভ জানার পর বর চ...,Positive,0.756598,0.726574,0.694848,0.211931,0.258446,0.333247,0.866512,0.094603,0.397723,0.875627,0.521611
1,image_ben_3908.jpg,কিন্তু মিষ্টতা নেই,Negative,0.158137,0.167763,0.198197,0.208470,0.258289,0.221382,0.079579,0.103506,0.252656,0.083301,0.173128
2,image_ben_4634.jpg,মানুষ যা দেখে প্রথম সিঙারা আবিষ্কারের ধারণা পায়,Positive,0.759429,0.742555,0.712545,0.699032,0.635443,0.693390,0.882138,0.905293,0.647585,0.875158,0.755257
3,image_ben_3192.png,এক মেয়ে ফ্যানে ওড়না পাচিয়ে সুই*সাইড করতে যাচ্ছ...,Negative,0.159403,0.180209,0.198676,0.208291,0.258787,0.221133,0.082621,0.077005,0.542590,0.086377,0.201509
4,image_ben_4836.jpg,No text detected,Negative,0.158329,0.168225,0.198251,0.208401,0.258507,0.220299,0.077872,0.122565,0.252441,0.079659,0.174455
5,image_ben_3907.jpg,বাঘের চামড়া দিয়ে বিড়াল ঢেকে রাখার একটি চিত্র।,Positive,0.707538,0.553200,0.206425,0.689686,0.258257,0.687782,0.857298,0.081603,0.542554,0.875199,0.545954
6,image_ben_4636.jpg,"আমার উপর রেগে আছো রিয়া? না, রেগে নেই তাহলে মুখ...",Negative,0.158023,0.168878,0.199360,0.208551,0.258904,0.219469,0.077874,0.074663,0.252714,0.077862,0.169630
7,image_ben_5080.jpg,নিঃস্ব উপেন তার দুই বিঘা জমি,Negative,0.158166,0.168277,0.197724,0.208321,0.258345,0.219638,0.077015,0.098507,0.257988,0.077395,0.172138
8,image_ben_3704.jpg,"চলো চিন্টু মজা হবে ! Yes, chumki তাড়াতাড়ি এসো ...",Negative,0.159320,0.168174,0.672235,0.208177,0.258125,0.219557,0.077326,0.078953,0.606166,0.077750,0.252578
9,image_ben_5071.jpg,No text detected,Negative,0.158329,0.168225,0.198251,0.208401,0.258507,0.220299,0.077872,0.122565,0.252441,0.079659,0.174455


In [19]:
subm_df.loc[(subm_df['avg'] <= 0.5) & (subm_df['task_1'] == 'Positive'), 'task_1'] = 'Neutral'

In [20]:
subm_df.head(20)

,Ids,OCR,task_1,y_0,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,avg
0,image_ben_1422.png,বৌ-এর প্রেগনেন্সি রিপোর্ট পজিটিভ জানার পর বর চ...,Positive,0.756598,0.726574,0.694848,0.211931,0.258446,0.333247,0.866512,0.094603,0.397723,0.875627,0.521611
1,image_ben_3908.jpg,কিন্তু মিষ্টতা নেই,Negative,0.158137,0.167763,0.198197,0.208470,0.258289,0.221382,0.079579,0.103506,0.252656,0.083301,0.173128
2,image_ben_4634.jpg,মানুষ যা দেখে প্রথম সিঙারা আবিষ্কারের ধারণা পায়,Positive,0.759429,0.742555,0.712545,0.699032,0.635443,0.693390,0.882138,0.905293,0.647585,0.875158,0.755257
3,image_ben_3192.png,এক মেয়ে ফ্যানে ওড়না পাচিয়ে সুই*সাইড করতে যাচ্ছ...,Negative,0.159403,0.180209,0.198676,0.208291,0.258787,0.221133,0.082621,0.077005,0.542590,0.086377,0.201509
4,image_ben_4836.jpg,No text detected,Negative,0.158329,0.168225,0.198251,0.208401,0.258507,0.220299,0.077872,0.122565,0.252441,0.079659,0.174455
5,image_ben_3907.jpg,বাঘের চামড়া দিয়ে বিড়াল ঢেকে রাখার একটি চিত্র।,Positive,0.707538,0.553200,0.206425,0.689686,0.258257,0.687782,0.857298,0.081603,0.542554,0.875199,0.545954
6,image_ben_4636.jpg,"আমার উপর রেগে আছো রিয়া? না, রেগে নেই তাহলে মুখ...",Negative,0.158023,0.168878,0.199360,0.208551,0.258904,0.219469,0.077874,0.074663,0.252714,0.077862,0.169630
7,image_ben_5080.jpg,নিঃস্ব উপেন তার দুই বিঘা জমি,Negative,0.158166,0.168277,0.197724,0.208321,0.258345,0.219638,0.077015,0.098507,0.257988,0.077395,0.172138
8,image_ben_3704.jpg,"চলো চিন্টু মজা হবে ! Yes, chumki তাড়াতাড়ি এসো ...",Negative,0.159320,0.168174,0.672235,0.208177,0.258125,0.219557,0.077326,0.078953,0.606166,0.077750,0.252578
9,image_ben_5071.jpg,No text detected,Negative,0.158329,0.168225,0.198251,0.208401,0.258507,0.220299,0.077872,0.122565,0.252441,0.079659,0.174455


In [21]:
subm_df[['Ids', params['label_col']]].to_csv('submission_task1.csv', index=False)

# TASK 2

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import logging
import random
from functools import partial
import sklearn
from sklearn.model_selection import StratifiedKFold
import datasets
import transformers
import torch
import copy
from torch import nn
import gc
import torch.nn.functional as F
from tqdm.auto import tqdm, trange
logger = logging.getLogger(__name__)

D:\MiniConda\envs\pythonProject18\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def set_seeds(self, seed=2023): 
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [4]:
# Source: https://huggingface.co/l3cube-pune/assamese-bert
params = dict(learning_rate=1.0e-5, seed=2023, data_seed=2023, n_splits=10,
              model_id='models--l3cube-pune--bengali-bert', tensorboard_dir='tensorboard', label_col='task_2',
              patience=5, metric='f1', metric_mode='max', train_batch_size=32, eval_batch_size=32,
              train_max_length=512, eval_max_length=512, learning_rate_scheduling='cosine_annealing',
              gradient_accumulation_steps=1, warmup_proportion=50, num_train_epochs=100, max_seq=512,
              fp16=False, logging_steps=None, model_dir='/model/', password='Bangla')

y_map = {0.0: 'Non-Sarcastic', 1.0: 'Sarcastic'}

In [5]:
set_seeds(params['seed'])

# Data Preparation

In [6]:
import getpass
import zipfile
from pathlib import Path
def prepare_data(path, password, train='Bangla_train_data.csv', test='bengali_test_data_wo_label.csv'):
    zfs = [zipfile.ZipFile(path / f.replace('.csv', '.zip')) for f in [train, test]]
    return [pd.read_csv(zfs[i].open(f, 'r', password)) for i, f in enumerate([train, test])]

def split(train_df, y, n_splits=5, data_seed=2023, shuffle=True):
    skf = StratifiedKFold(n_splits=n_splits, random_state=data_seed, shuffle=shuffle)
    train_df['split'] = -1
    for i, (train_index, test_index) in enumerate(skf.split(train_df.index, y)):
        train_df.loc[train_df.index.isin(test_index), 'split'] = i
    return train_df        

path = Path('Bangla')
password =  bytes(getpass.getpass(f"Input file password") if 'password' not in params else params['password'], 'utf-8')
train_df, test_df = prepare_data(path, password=password)
train_df = split(train_df, train_df['Sarcasm'], n_splits=params['n_splits'], data_seed=params['data_seed'])
train_df['task_2'] = train_df['Sarcasm'].map({v: k for k, v in y_map.items()})
print(len(train_df), len(test_df))

2693 1821


In [7]:
def prepare_feature(df, y_col, tokenizer, max_length):
    texts = df['OCR'].astype(str).tolist()
    features = tokenizer(texts, padding=False, truncation=True, return_tensors=None, max_length=max_length).data
    if y_col in df.columns:
        features['labels'] = df[y_col].tolist()
    return features

In [8]:
train_df.head()

,Ids,Sentiment,Sarcasm,Vulgar,Abuse,Target,OCR,split,task_2
0,image_ben_3635.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,I don't need any fucking GIRL বাঁরা কি সাইলেন...,4,1.0
1,image_ben_1586.png,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"After a little problem in relationship Boys, G...",0,1.0
2,image_ben_4040.jpg,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"মা জানো, পাশের বাড়ির চুলবুলি আমার থেকে বেশি অ...",6,1.0
3,image_ben_5074.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,Gf কে ফিঙ্গারং করে দেওয়ার পর যখন সে চরমসুখ অনু...,0,1.0
4,image_ben_7351.jpg,Negative,Sarcastic,Vulgar,Abusive,Gender,টিপতে টিপতে যখন ব্যাথ্যা অনুভব হয়,2,1.0


# Prepare Transformer Trainer

In [9]:
class Model(nn.Module):
    def __init__(self, transformer):
        super().__init__()
        self.transformer = transformer 
        
    def m_loss(self, logits, labels):
        return F.binary_cross_entropy_with_logits(logits.squeeze(-1), labels)
        
    def forward(self, **kwargs):
        labels = kwargs.pop("labels", None)
        output = self.transformer(**kwargs)
        output.loss = self.m_loss(output.logits, labels) if labels != None else output.loss
        output.logits = F.sigmoid(output.logits)
        return output
    
    def load(self, model_dir):
        self.transformer.from_pretrained(model_dir)
        print(f"Model loaded: {model_dir}")
    
    def save(self, model_dir):
        self.transformer.save_pretrained(model_dir)
    
class PTTrainer(transformers.Trainer):
    def __init__(self, **kwargs): 
        super().__init__(**kwargs)
        self.compute_metrics = self._metrics if self.eval_dataset is not None else None
        self._patience = 0

    def _metrics(self, eval_preds):
        """
        Evaluation metrics
        """
        threshold = 0.5
        thresholds = np.arange(0.01, 1.0, 0.01) 
        scores, labels = eval_preds
        acc, f1, thr = -1, -1, 0
        auc = sklearn.metrics.roc_auc_score(labels, scores)
        for threshold in thresholds:
            predictions = (scores >= threshold).astype(int)
            new_f1 = sklearn.metrics.f1_score(labels, predictions, average='macro')
            if new_f1 > f1:
                acc = sklearn.metrics.accuracy_score(labels, predictions)
                f1 = new_f1
                thr = threshold
        return {'auc': auc, 'accuracy': acc, 'f1': f1, 'thr': thr}
        
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        forward/backward step
        """
        output = model(**inputs)
        return (output.loss, {'logits': output.logits}) if return_outputs else output.loss

    def _save_checkpoint(self, model, trial, metrics=None):
        # Determine the new best metric / best model checkpoint
        if metrics is not None and self.args.metric_for_best_model is not None:
            metric_to_check = self.args.metric_for_best_model
            if not metric_to_check.startswith("eval_"):
                metric_to_check = f"eval_{metric_to_check}"
            metric_value = metrics[metric_to_check]

            operator = np.greater if self.args.greater_is_better else np.less
            if (
                self.state.best_metric is None
                # or self.state.best_model_checkpoint is None
                or operator(metric_value, self.state.best_metric)
            ):
                self.state.best_metric = metric_value
                # self.state.best_model_checkpoint = self.args.output_dir
                model.save(self.args.output_dir)
                print(f'Model Saved: {metric_value}')
                self._patience = 0
            else:
                print(f"No improvments: {metric_value}, patience: {self._patience}, lr: {self.optimizer.param_groups[0]['lr']}")
                self._patience += 1
            # LR annealing
            if params['learning_rate_scheduling'].lower() == 'cosine_annealing':
                if (self._patience - 1) == int(round(params['patience'] / 2)):
                    if not isinstance(self.lr_scheduler, torch.optim.lr_scheduler.CosineAnnealingLR):
                        self.lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=10, eta_min=max(self.args.learning_rate * 0.01, 1.0e-7), verbose=False)
                        print(f'Starting CosineAnnealingLR schedular.')
                        self._patience -= 1
        else:
            model.save(self.args.output_dir)
            print('Model Saved')


class PTDefaultFlowCallback(transformers.DefaultFlowCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        control = super().on_epoch_end(args, state, control, **kwargs)
        # Evaluate
        if args.evaluation_strategy != transformers.IntervalStrategy.EPOCH \
            and state.epoch == args.num_train_epochs:
            control.should_evaluate = state.best_metric is not None
            control.should_save = True
        return control
    
def get_best_thr(history):
    max_i = -1
    for i, record in enumerate(history):
        if f"eval_{params['metric']}" in record:
            if max_i == -1 or record[f"eval_{params['metric']}"] > history[max_i][f"eval_{params['metric']}"]:
                max_i = i
    return history[max_i]['eval_thr'], history[max_i]['eval_f1']

def apply_thr(y, threshold, mapper=None):
    if mapper is None:
        return (y > threshold).astype(int)
    else:
        return [mapper[x] for x in (y > threshold).astype(int)]

def inference(pred_features, threshold=None):
    print(f"Threshold: {threshold}")
    result = trainer.predict(datasets.Dataset.from_dict(dict(pred_features))).predictions
    if threshold is None:
        test_df[params['label_col']] = result[:, 0]
    else:
        test_df[params['label_col']] = apply_thr(result[:, 0], threshold, y_map)
    return test_df[[col for col in test_df.columns if col != 'text']]

In [10]:
transformer = transformers.AutoModelForSequenceClassification.from_pretrained(params['model_id'], num_labels=1)
_tokenizer = transformers.AutoTokenizer.from_pretrained(params['model_id'], model_max_length=params['max_seq'])
callbacks = [PTDefaultFlowCallback(), 
             transformers.EarlyStoppingCallback(early_stopping_patience=params['patience'])]

if isinstance(params['logging_steps'], int):
    save_strategy = transformers.IntervalStrategy.STEPS
else:
    save_strategy = transformers.IntervalStrategy.EPOCH 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at models--l3cube-pune--bengali-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def get_trainer(valid_split=0):
    _model = Model(copy.deepcopy(transformer))
    optimizer = torch.optim.AdamW(_model.parameters(), lr=params['learning_rate'])
    model_dir = f"{params['model_dir']}{valid_split}"
    t_df, v_df = train_df[train_df['split'] != valid_split], train_df[train_df['split'] == valid_split]
    print(f"Train: {len(t_df)}, Valid: {len(v_df)}")
    train_features = prepare_feature(t_df, params['label_col'], _tokenizer, max_length=params['train_max_length'])
    valid_features = prepare_feature(v_df, params['label_col'], _tokenizer, max_length=params['train_max_length'])
    trainer = PTTrainer(model=_model,
        train_dataset=datasets.Dataset.from_dict(dict(train_features)),
        eval_dataset=datasets.Dataset.from_dict(dict(valid_features)) if valid_features is not None else None,
        args=transformers.TrainingArguments(
            label_names=['labels'],
            seed=params['seed'],
            data_seed=params['data_seed'],
            remove_unused_columns=False,
            report_to="tensorboard" if params['tensorboard_dir'] is not None else "none",
            logging_dir=params['tensorboard_dir'],
            per_device_train_batch_size=params['train_batch_size'], 
            per_device_eval_batch_size=params['eval_batch_size'],
            gradient_accumulation_steps=params['gradient_accumulation_steps'],
            warmup_steps=params['warmup_proportion'], 
            # max_steps=20,
            num_train_epochs=params['num_train_epochs'],
            learning_rate=params['learning_rate'], 
            fp16=params['fp16'],
            metric_for_best_model=params['metric'],
            # gradient_checkpointing=fp16,
            # evaluate_during_training=True,
            logging_steps=params['logging_steps'], 
            output_dir=model_dir,
            evaluation_strategy=save_strategy,
            save_strategy=save_strategy,
            logging_strategy=save_strategy,
            save_total_limit=1,
            greater_is_better=params['metric_mode'], 
            load_best_model_at_end=True,
            eval_steps=params['logging_steps'],
            save_steps=params['logging_steps'],
            overwrite_output_dir=True,
            max_grad_norm=1.0,
        ),
        optimizers=[optimizer, transformers.get_constant_schedule(optimizer)],
        callbacks=callbacks,
        data_collator=transformers.DataCollatorWithPadding(_tokenizer),
    )
    return _model, trainer, model_dir

# Train / Inference

In [13]:
pred_features = prepare_feature(test_df, params['label_col'], _tokenizer, max_length=params['eval_max_length'])
subm_df = None
thresholds = []
scores = []
cols = []
for fold in range(params['n_splits']):
    print(f"Fold: {fold + 1} out of {params['n_splits']}")
    col = f'y_{fold}'
    model, trainer, model_dir = get_trainer(fold)
    trainer.train()
    trainer.model.load(model_dir)
    subm = inference(pred_features)
    t, f1 = get_best_thr(trainer.state.log_history)
    thresholds.append(t)
    scores.append(f1)
    if subm_df is None:
        subm_df = subm
    subm_df[col] = subm[params['label_col']]
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    cols.append(col)

Fold: 1 out of 10
Train: 2423, Valid: 270


D:\MiniConda\envs\pythonProject18\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Auc,Accuracy,F1,Thr
1,0.658300,0.621271,0.565417,0.737037,0.450323,0.560000
2,0.603100,0.586219,0.487019,0.725926,0.488270,0.630000
3,0.568600,0.560498,0.532199,0.737037,0.450323,0.560000
4,0.544500,0.546546,0.556789,0.744444,0.488454,0.700000
5,0.529500,0.539703,0.523727,0.737037,0.494609,0.760000
6,0.517500,0.539935,0.476547,0.733333,0.492481,0.760000
7,0.504500,0.545379,0.561573,0.711111,0.521364,0.800000
8,0.488700,0.518194,0.606401,0.729630,0.606579,0.590000
9,0.463700,0.532188,0.652796,0.729630,0.611235,0.830000
10,0.436400,0.542779,0.598870,0.740741,0.583186,0.820000


Model Saved: 0.4503225806451613
Model Saved: 0.4882696445036369
No improvments: 0.4503225806451613, patience: 0, lr: 1e-05
Model Saved: 0.48845382904527856
Model Saved: 0.49460863146239226
No improvments: 0.49248120300751874, patience: 0, lr: 1e-05
Model Saved: 0.5213636363636364
Model Saved: 0.6065789736322083
Model Saved: 0.6112349356003077
No improvments: 0.5831863091037403, patience: 0, lr: 1e-05
Model Saved: 0.6290106951871658
No improvments: 0.6126568466993999, patience: 0, lr: 1e-05
No improvments: 0.536303787806507, patience: 1, lr: 1e-05
No improvments: 0.5634095634095634, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.6201096201096201, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.6039408866995074, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/0
Threshold: None


Fold: 2 out of 10
Train: 2423, Valid: 270


...

Model Saved: 0.4360587002096436
Model Saved: 0.5225793729047525
No improvments: 0.4360587002096436, patience: 0, lr: 1e-05
Model Saved: 0.547475248889992
Model Saved: 0.5786754564368704
No improvments: 0.5539687633718442, patience: 0, lr: 1e-05
No improvments: 0.5294299238526445, patience: 1, lr: 1e-05
No improvments: 0.5432702202099198, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.5406631495782043, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.5251925545571245, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/8
Threshold: None


Fold: 10 out of 10
Train: 2424, Valid: 269


D:\MiniConda\envs\pythonProject18\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Auc,Accuracy,F1,Thr
1,0.659200,0.621044,0.640211,0.754647,0.515448,0.570000
2,0.604400,0.581998,0.645846,0.754647,0.525193,0.630000
3,0.570700,0.555202,0.657393,0.773234,0.436059,0.010000
4,0.550700,0.540043,0.614518,0.747212,0.538081,0.700000
5,0.537300,0.533872,0.657432,0.754647,0.515448,0.580000
6,0.529300,0.531947,0.538422,0.754647,0.515448,0.590000
7,0.528700,0.528236,0.656447,0.710037,0.595816,0.780000
8,0.526600,0.532769,0.526403,0.750929,0.513116,0.600000
9,0.526800,0.531552,0.628941,0.754647,0.525193,0.780000
10,0.520200,0.517353,0.627916,0.732342,0.609201,0.660000


Model Saved: 0.5154475982532751
Model Saved: 0.5251925545571245
No improvments: 0.4360587002096436, patience: 0, lr: 1e-05
Model Saved: 0.538080808080808
No improvments: 0.5154475982532751, patience: 0, lr: 1e-05
No improvments: 0.5154475982532751, patience: 1, lr: 1e-05
Model Saved: 0.5958159963014331
No improvments: 0.5131155955371856, patience: 0, lr: 1e-05
No improvments: 0.5251925545571245, patience: 1, lr: 1e-05
Model Saved: 0.6092009685230024
Model Saved: 0.6166163648558015
No improvments: 0.5794684731631058, patience: 0, lr: 1e-05
No improvments: 0.5799062988027069, patience: 1, lr: 1e-05
No improvments: 0.5933567576401373, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.5977457873005245, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.5589640014256861, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/9
Threshold: None


In [14]:
thresholds, scores

([0.85,
  0.81,
  0.63,
  0.9400000000000001,
  0.78,
  0.77,
  0.93,
  0.88,
  0.7100000000000001,
  0.8],
 [0.6290106951871658,
  0.6460887949260042,
  0.6881496881496881,
  0.6526230831315577,
  0.6038471124000272,
  0.6185749734136831,
  0.6237906393087725,
  0.6817263410816026,
  0.5786754564368704,
  0.6166163648558015])

In [15]:
thr = np.array(thresholds).mean()
print(f"Best threshold: {thr}")
print(f"Best score: {np.array(scores).mean()}")
subm_df['avg'] = subm_df[cols].apply('mean', axis=1)
subm_df[params['label_col']] = apply_thr(subm_df['avg'], thr, y_map)

Best threshold: 0.8100000000000002
Best score: 0.6339103148891172


In [16]:
subm_df

,Ids,OCR,task_2,y_0,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,avg
0,image_ben_1422.png,বৌ-এর প্রেগনেন্সি রিপোর্ট পজিটিভ জানার পর বর চ...,Sarcastic,0.920285,0.884400,0.890145,0.968584,0.863905,0.866582,0.947374,0.917455,0.813932,0.878986,0.895165
1,image_ben_3908.jpg,কিন্তু মিষ্টতা নেই,Non-Sarcastic,0.920171,0.884323,0.889996,0.968575,0.863596,0.280780,0.947541,0.180733,0.813750,0.878866,0.762833
2,image_ben_4634.jpg,মানুষ যা দেখে প্রথম সিঙারা আবিষ্কারের ধারণা পায়,Sarcastic,0.920496,0.884416,0.890133,0.966867,0.601943,0.866519,0.946761,0.918940,0.813926,0.878867,0.868887
3,image_ben_3192.png,এক মেয়ে ফ্যানে ওড়না পাচিয়ে সুই*সাইড করতে যাচ্ছ...,Sarcastic,0.912886,0.884368,0.890058,0.870699,0.863367,0.866702,0.943914,0.917021,0.813937,0.879009,0.884196
4,image_ben_4836.jpg,No text detected,Non-Sarcastic,0.200692,0.597275,0.516847,0.587422,0.508416,0.545424,0.550833,0.540903,0.418527,0.623912,0.509025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,image_ben_7561.jpg,বিদায় লেজেন্ড ফুটবল খেলা দেখার মজা টাই হয়ত হার...,Sarcastic,0.920412,0.884454,0.890100,0.968359,0.863496,0.867102,0.947377,0.918897,0.813995,0.878642,0.895284
1817,image_ben_3223.png,শীতে ঘটিদের পানীয় শীতে বাঙালদের পানীয়,Sarcastic,0.920468,0.884379,0.890132,0.968765,0.864004,0.866863,0.947568,0.918957,0.813825,0.878208,0.895317
1818,image_ben_3254.png,ঘটি বাঙীল ভাষায় খিস্তি দিয়ে একে বাংলা করে দে...,Sarcastic,0.920408,0.884412,0.890129,0.968877,0.863928,0.866756,0.947464,0.918890,0.813946,0.878511,0.895332
1819,image_ben_74.png,FAU-G FEARLESS AND UNITED GUARDS,Non-Sarcastic,0.919542,0.879495,0.248819,0.072251,0.863730,0.859202,0.109765,0.180740,0.813701,0.257382,0.520463


In [17]:
subm_df[['Ids', params['label_col']]].to_csv('submission_task2.csv', index=False)

# TASK 3

In [57]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import logging
import random
from functools import partial
import sklearn
from sklearn.model_selection import StratifiedKFold
import datasets
import transformers
import torch
import copy
from torch import nn
import gc
import torch.nn.functional as F
from tqdm.auto import tqdm, trange
logger = logging.getLogger(__name__)

In [58]:
def set_seeds(self, seed=2023): 
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [59]:
# Source: https://huggingface.co/l3cube-pune/assamese-bert
params = dict(learning_rate=1.0e-5, seed=2023, data_seed=2023, n_splits=10,
              model_id='models--l3cube-pune--bengali-bert', tensorboard_dir='tensorboard', label_col='task_3',
              patience=5, metric='f1', metric_mode='max', train_batch_size=32, eval_batch_size=32,
              train_max_length=512, eval_max_length=512, learning_rate_scheduling='cosine_annealing',
              gradient_accumulation_steps=1, warmup_proportion=50, num_train_epochs=100, max_seq=512,
              fp16=False, logging_steps=None, model_dir='/model/', password='Bangla')

y_map = {0.0: 'Non Vulgar', 1.0: 'Vulgar'}

In [60]:
set_seeds(params['seed'])

# Data Preparation

In [61]:
import getpass
import zipfile
from pathlib import Path
def prepare_data(path, password, train='Bangla_train_data.csv', test='bengali_test_data_wo_label.csv'):
    zfs = [zipfile.ZipFile(path / f.replace('.csv', '.zip')) for f in [train, test]]
    return [pd.read_csv(zfs[i].open(f, 'r', password)) for i, f in enumerate([train, test])]

def split(train_df, y, n_splits=5, data_seed=2023, shuffle=True):
    skf = StratifiedKFold(n_splits=n_splits, random_state=data_seed, shuffle=shuffle)
    train_df['split'] = -1
    for i, (train_index, test_index) in enumerate(skf.split(train_df.index, y)):
        train_df.loc[train_df.index.isin(test_index), 'split'] = i
    return train_df        

path = Path('Bangla')
password =  bytes(getpass.getpass(f"Input file password") if 'password' not in params else params['password'], 'utf-8')
train_df, test_df = prepare_data(path, password=password)
train_df = split(train_df, train_df['Vulgar'], n_splits=params['n_splits'], data_seed=params['data_seed'])
train_df['task_3'] = train_df['Vulgar'].map({v: k for k, v in y_map.items()})
print(len(train_df), len(test_df))

2693 1821


In [62]:
def prepare_feature(df, y_col, tokenizer, max_length):
    texts = df['OCR'].astype(str).tolist()
    features = tokenizer(texts, padding=False, truncation=True, return_tensors=None, max_length=max_length).data
    if y_col in df.columns:
        features['labels'] = df[y_col].tolist()
    return features

In [63]:
train_df.head()

,Ids,Sentiment,Sarcasm,Vulgar,Abuse,Target,OCR,split,task_3
0,image_ben_3635.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,I don't need any fucking GIRL বাঁরা কি সাইলেন...,1,0.0
1,image_ben_1586.png,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"After a little problem in relationship Boys, G...",6,0.0
2,image_ben_4040.jpg,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"মা জানো, পাশের বাড়ির চুলবুলি আমার থেকে বেশি অ...",1,0.0
3,image_ben_5074.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,Gf কে ফিঙ্গারং করে দেওয়ার পর যখন সে চরমসুখ অনু...,3,0.0
4,image_ben_7351.jpg,Negative,Sarcastic,Vulgar,Abusive,Gender,টিপতে টিপতে যখন ব্যাথ্যা অনুভব হয়,0,1.0


# Prepare Transformer Trainer

In [64]:
class Model(nn.Module):
    def __init__(self, transformer):
        super().__init__()
        self.transformer = transformer 
        
    def m_loss(self, logits, labels):
        return F.binary_cross_entropy_with_logits(logits.squeeze(-1), labels)
        
    def forward(self, **kwargs):
        labels = kwargs.pop("labels", None)
        output = self.transformer(**kwargs)
        output.loss = self.m_loss(output.logits, labels) if labels != None else output.loss
        output.logits = F.sigmoid(output.logits)
        return output
    
    def load(self, model_dir):
        self.transformer.from_pretrained(model_dir)
        print(f"Model loaded: {model_dir}")
    
    def save(self, model_dir):
        self.transformer.save_pretrained(model_dir)
    
class PTTrainer(transformers.Trainer):
    def __init__(self, **kwargs): 
        super().__init__(**kwargs)
        self.compute_metrics = self._metrics if self.eval_dataset is not None else None
        self._patience = 0

    def _metrics(self, eval_preds):
        """
        Evaluation metrics
        """
        threshold = 0.5
        thresholds = np.arange(0.01, 1.0, 0.01) 
        scores, labels = eval_preds
        acc, f1, thr = -1, -1, 0
        auc = sklearn.metrics.roc_auc_score(labels, scores)
        for threshold in thresholds:
            predictions = (scores >= threshold).astype(int)
            new_f1 = sklearn.metrics.f1_score(labels, predictions, average='macro')
            if new_f1 > f1:
                acc = sklearn.metrics.accuracy_score(labels, predictions)
                f1 = new_f1
                thr = threshold
        return {'auc': auc, 'accuracy': acc, 'f1': f1, 'thr': thr}
        
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        forward/backward step
        """
        output = model(**inputs)
        return (output.loss, {'logits': output.logits}) if return_outputs else output.loss

    def _save_checkpoint(self, model, trial, metrics=None):
        # Determine the new best metric / best model checkpoint
        if metrics is not None and self.args.metric_for_best_model is not None:
            metric_to_check = self.args.metric_for_best_model
            if not metric_to_check.startswith("eval_"):
                metric_to_check = f"eval_{metric_to_check}"
            metric_value = metrics[metric_to_check]

            operator = np.greater if self.args.greater_is_better else np.less
            if (
                self.state.best_metric is None
                # or self.state.best_model_checkpoint is None
                or operator(metric_value, self.state.best_metric)
            ):
                self.state.best_metric = metric_value
                # self.state.best_model_checkpoint = self.args.output_dir
                model.save(self.args.output_dir)
                print(f'Model Saved: {metric_value}')
                self._patience = 0
            else:
                print(f"No improvments: {metric_value}, patience: {self._patience}, lr: {self.optimizer.param_groups[0]['lr']}")
                self._patience += 1
            # LR annealing
            if params['learning_rate_scheduling'].lower() == 'cosine_annealing':
                if (self._patience - 1) == int(round(params['patience'] / 2)):
                    if not isinstance(self.lr_scheduler, torch.optim.lr_scheduler.CosineAnnealingLR):
                        self.lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=10, eta_min=max(self.args.learning_rate * 0.01, 1.0e-7), verbose=False)
                        print(f'Starting CosineAnnealingLR schedular.')
                        self._patience -= 1
        else:
            model.save(self.args.output_dir)
            print('Model Saved')


class PTDefaultFlowCallback(transformers.DefaultFlowCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        control = super().on_epoch_end(args, state, control, **kwargs)
        # Evaluate
        if args.evaluation_strategy != transformers.IntervalStrategy.EPOCH \
            and state.epoch == args.num_train_epochs:
            control.should_evaluate = state.best_metric is not None
            control.should_save = True
        return control
    
def get_best_thr(history):
    max_i = -1
    for i, record in enumerate(history):
        if f"eval_{params['metric']}" in record:
            if max_i == -1 or record[f"eval_{params['metric']}"] > history[max_i][f"eval_{params['metric']}"]:
                max_i = i
    return history[max_i]['eval_thr'], history[max_i]['eval_f1']

def apply_thr(y, threshold, mapper=None):
    if mapper is None:
        return (y > threshold).astype(int)
    else:
        return [mapper[x] for x in (y > threshold).astype(int)]

def inference(pred_features, threshold=None):
    print(f"Threshold: {threshold}")
    result = trainer.predict(datasets.Dataset.from_dict(dict(pred_features))).predictions
    if threshold is None:
        test_df[params['label_col']] = result[:, 0]
    else:
        test_df[params['label_col']] = apply_thr(result[:, 0], threshold, y_map)
    return test_df[[col for col in test_df.columns if col != 'text']]

In [65]:
transformer = transformers.AutoModelForSequenceClassification.from_pretrained(params['model_id'], num_labels=1)
_tokenizer = transformers.AutoTokenizer.from_pretrained(params['model_id'], model_max_length=params['max_seq'])
callbacks = [PTDefaultFlowCallback(), 
             transformers.EarlyStoppingCallback(early_stopping_patience=params['patience'])]

if isinstance(params['logging_steps'], int):
    save_strategy = transformers.IntervalStrategy.STEPS
else:
    save_strategy = transformers.IntervalStrategy.EPOCH 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at models--l3cube-pune--bengali-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
def get_trainer(valid_split=0):
    _model = Model(copy.deepcopy(transformer))
    optimizer = torch.optim.AdamW(_model.parameters(), lr=params['learning_rate'])
    model_dir = f"{params['model_dir']}{valid_split}"
    t_df, v_df = train_df[train_df['split'] != valid_split], train_df[train_df['split'] == valid_split]
    print(f"Train: {len(t_df)}, Valid: {len(v_df)}")
    train_features = prepare_feature(t_df, params['label_col'], _tokenizer, max_length=params['train_max_length'])
    valid_features = prepare_feature(v_df, params['label_col'], _tokenizer, max_length=params['train_max_length'])
    trainer = PTTrainer(model=_model,
        train_dataset=datasets.Dataset.from_dict(dict(train_features)),
        eval_dataset=datasets.Dataset.from_dict(dict(valid_features)) if valid_features is not None else None,
        args=transformers.TrainingArguments(
            label_names=['labels'],
            seed=params['seed'],
            data_seed=params['data_seed'],
            remove_unused_columns=False,
            report_to="tensorboard" if params['tensorboard_dir'] is not None else "none",
            logging_dir=params['tensorboard_dir'],
            per_device_train_batch_size=params['train_batch_size'], 
            per_device_eval_batch_size=params['eval_batch_size'],
            gradient_accumulation_steps=params['gradient_accumulation_steps'],
            warmup_steps=params['warmup_proportion'], 
            # max_steps=20,
            num_train_epochs=params['num_train_epochs'],
            learning_rate=params['learning_rate'], 
            fp16=params['fp16'],
            metric_for_best_model=params['metric'],
            # gradient_checkpointing=fp16,
            # evaluate_during_training=True,
            logging_steps=params['logging_steps'], 
            output_dir=model_dir,
            evaluation_strategy=save_strategy,
            save_strategy=save_strategy,
            logging_strategy=save_strategy,
            save_total_limit=1,
            greater_is_better=params['metric_mode'], 
            load_best_model_at_end=True,
            eval_steps=params['logging_steps'],
            save_steps=params['logging_steps'],
            overwrite_output_dir=True,
            max_grad_norm=1.0,
        ),
        optimizers=[optimizer, transformers.get_constant_schedule(optimizer)],
        callbacks=callbacks,
        data_collator=transformers.DataCollatorWithPadding(_tokenizer),
    )
    return _model, trainer, model_dir

# Train / Inference

In [67]:
pred_features = prepare_feature(test_df, params['label_col'], _tokenizer, max_length=params['eval_max_length'])
subm_df = None
thresholds = []
scores = []
cols = []
for fold in range(params['n_splits']):
    print(f"Fold: {fold + 1} out of {params['n_splits']}")
    col = f'y_{fold}'
    model, trainer, model_dir = get_trainer(fold)
    trainer.train()
    trainer.model.load(model_dir)
    subm = inference(pred_features)
    t, f1 = get_best_thr(trainer.state.log_history)
    thresholds.append(t)
    scores.append(f1)
    if subm_df is None:
        subm_df = subm
    subm_df[col] = subm[params['label_col']]
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    cols.append(col)

Fold: 1 out of 10
Train: 2423, Valid: 270


D:\MiniConda\envs\pythonProject18\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Auc,Accuracy,F1,Thr
1,0.654700,0.609863,0.477006,0.825926,0.452333,0.440000
2,0.584900,0.555144,0.455539,0.825926,0.452333,0.380000
3,0.535000,0.510675,0.512117,0.825926,0.452333,0.320000
4,0.496900,0.482621,0.493369,0.825926,0.452333,0.260000
5,0.479200,0.474312,0.485784,0.825926,0.452333,0.240000
6,0.471700,0.469238,0.470613,0.825926,0.452333,0.230000


Model Saved: 0.45233265720081134
No improvments: 0.45233265720081134, patience: 0, lr: 1e-05
No improvments: 0.45233265720081134, patience: 1, lr: 1e-05
No improvments: 0.45233265720081134, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.45233265720081134, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.45233265720081134, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/0
Threshold: None


Fold: 2 out of 10
Train: 2423, Valid: 270


...

Model Saved: 0.45213849287169044
No improvments: 0.45213849287169044, patience: 0, lr: 1e-05
No improvments: 0.45213849287169044, patience: 1, lr: 1e-05
No improvments: 0.45213849287169044, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.45213849287169044, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.45213849287169044, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/8
Threshold: None


Fold: 10 out of 10
Train: 2424, Valid: 269


D:\MiniConda\envs\pythonProject18\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Auc,Accuracy,F1,Thr
1,0.655200,0.610614,0.415181,0.825279,0.472351,0.430000
2,0.585700,0.556068,0.464827,0.825279,0.452138,0.380000
3,0.535100,0.511486,0.441441,0.825279,0.452138,0.320000
4,0.497400,0.483309,0.438662,0.825279,0.452138,0.260000
5,0.479200,0.475287,0.519791,0.825279,0.452138,0.240000
6,0.472100,0.469609,0.464922,0.825279,0.452138,0.230000


Model Saved: 0.4723509035516047
No improvments: 0.45213849287169044, patience: 0, lr: 1e-05
No improvments: 0.45213849287169044, patience: 1, lr: 1e-05
No improvments: 0.45213849287169044, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.45213849287169044, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.45213849287169044, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/9
Threshold: None


In [68]:
thresholds, scores

([0.44, 0.37, 0.44, 0.43, 0.43, 0.44, 0.44, 0.38, 0.44, 0.43],
 [0.45233265720081134,
  0.5240121580547112,
  0.45233265720081134,
  0.49755747126436783,
  0.5100182149362477,
  0.4532520325203252,
  0.45213849287169044,
  0.4738945578231292,
  0.45213849287169044,
  0.4723509035516047])

In [71]:
thr = np.array(thresholds).mean()
thr = 0.2146
print(f"Best threshold: {thr}")
print(f"Best score: {np.array(scores).mean()}")
subm_df['avg'] = subm_df[cols].apply('mean', axis=1)
subm_df[params['label_col']] = apply_thr(subm_df['avg'], thr, y_map)

Best threshold: 0.2146
Best score: 0.47400276382953893


In [72]:
subm_df.head(40)

,Ids,OCR,task_3,y_0,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,avg
0,image_ben_1422.png,বৌ-এর প্রেগনেন্সি রিপোর্ট পজিটিভ জানার পর বর চ...,Non Vulgar,0.221593,0.195037,0.218244,0.219149,0.219180,0.219178,0.219324,0.195029,0.218782,0.220010,0.214553
1,image_ben_3908.jpg,কিন্তু মিষ্টতা নেই,Vulgar,0.221780,0.194966,0.218370,0.219276,0.219295,0.219177,0.219385,0.195067,0.218859,0.220065,0.214624
2,image_ben_4634.jpg,মানুষ যা দেখে প্রথম সিঙারা আবিষ্কারের ধারণা পায়,Non Vulgar,0.221655,0.194940,0.218267,0.219130,0.219249,0.219151,0.219355,0.195031,0.218801,0.220033,0.214561
3,image_ben_3192.png,এক মেয়ে ফ্যানে ওড়না পাচিয়ে সুই*সাইড করতে যাচ্ছ...,Non Vulgar,0.221635,0.195004,0.218224,0.219124,0.219188,0.219280,0.219356,0.195050,0.218775,0.220030,0.214566
4,image_ben_4836.jpg,No text detected,Vulgar,0.221819,0.194996,0.218351,0.219234,0.219261,0.219666,0.219400,0.195322,0.218937,0.220051,0.214704
5,image_ben_3907.jpg,বাঘের চামড়া দিয়ে বিড়াল ঢেকে রাখার একটি চিত্র।,Non Vulgar,0.221684,0.194964,0.218251,0.219191,0.219195,0.219190,0.219363,0.195082,0.218821,0.220068,0.214581
6,image_ben_4636.jpg,"আমার উপর রেগে আছো রিয়া? না, রেগে নেই তাহলে মুখ...",Non Vulgar,0.221607,0.195057,0.218236,0.219135,0.219174,0.219148,0.219341,0.195037,0.218763,0.220014,0.214551
7,image_ben_5080.jpg,নিঃস্ব উপেন তার দুই বিঘা জমি,Non Vulgar,0.221690,0.194969,0.218282,0.219185,0.219215,0.219257,0.219352,0.195047,0.218842,0.220017,0.214586
8,image_ben_3704.jpg,"চলো চিন্টু মজা হবে ! Yes, chumki তাড়াতাড়ি এসো ...",Non Vulgar,0.221622,0.195075,0.218252,0.219151,0.219195,0.219144,0.219354,0.195027,0.218759,0.220021,0.214560
9,image_ben_5071.jpg,No text detected,Vulgar,0.221819,0.194996,0.218351,0.219234,0.219261,0.219666,0.219400,0.195322,0.218937,0.220051,0.214704


In [73]:
subm_df[['Ids', params['label_col']]].to_csv('submission_task3.csv', index=False)

# TASK 4

In [39]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import logging
import random
from functools import partial
import sklearn
from sklearn.model_selection import StratifiedKFold
import datasets
import transformers
import torch
import copy
from torch import nn
import gc
import torch.nn.functional as F
from tqdm.auto import tqdm, trange
logger = logging.getLogger(__name__)

In [40]:
def set_seeds(self, seed=2023): 
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [41]:
# Source: https://huggingface.co/l3cube-pune/assamese-bert
params = dict(learning_rate=1.0e-5, seed=2023, data_seed=2023, n_splits=10,
              model_id='models--l3cube-pune--bengali-bert', tensorboard_dir='tensorboard', label_col='task_4',
              patience=5, metric='f1', metric_mode='max', train_batch_size=32, eval_batch_size=32,
              train_max_length=512, eval_max_length=512, learning_rate_scheduling='cosine_annealing',
              gradient_accumulation_steps=1, warmup_proportion=50, num_train_epochs=100, max_seq=512,
              fp16=False, logging_steps=None, model_dir='/model/', password='Bangla')

y_map = {0.0: 'Non-abusive', 1.0: 'Abusive'}

In [42]:
set_seeds(params['seed'])

# Data Preparation

In [43]:
import getpass
import zipfile
from pathlib import Path
def prepare_data(path, password, train='Bangla_train_data.csv', test='bengali_test_data_wo_label.csv'):
    zfs = [zipfile.ZipFile(path / f.replace('.csv', '.zip')) for f in [train, test]]
    return [pd.read_csv(zfs[i].open(f, 'r', password)) for i, f in enumerate([train, test])]

def split(train_df, y, n_splits=5, data_seed=2023, shuffle=True):
    skf = StratifiedKFold(n_splits=n_splits, random_state=data_seed, shuffle=shuffle)
    train_df['split'] = -1
    for i, (train_index, test_index) in enumerate(skf.split(train_df.index, y)):
        train_df.loc[train_df.index.isin(test_index), 'split'] = i
    return train_df        

path = Path('Bangla')
password =  bytes(getpass.getpass(f"Input file password") if 'password' not in params else params['password'], 'utf-8')
train_df, test_df = prepare_data(path, password=password)
train_df = split(train_df, train_df['Abuse'], n_splits=params['n_splits'], data_seed=params['data_seed'])
train_df['task_4'] = train_df['Abuse'].map({v: k for k, v in y_map.items()})
print(len(train_df), len(test_df))

2693 1821


In [44]:
def prepare_feature(df, y_col, tokenizer, max_length):
    texts = df['OCR'].astype(str).tolist()
    features = tokenizer(texts, padding=False, truncation=True, return_tensors=None, max_length=max_length).data
    if y_col in df.columns:
        features['labels'] = df[y_col].tolist()
    return features

In [45]:
train_df.head()

,Ids,Sentiment,Sarcasm,Vulgar,Abuse,Target,OCR,split,task_4
0,image_ben_3635.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,I don't need any fucking GIRL বাঁরা কি সাইলেন...,2,1.0
1,image_ben_1586.png,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"After a little problem in relationship Boys, G...",8,1.0
2,image_ben_4040.jpg,Negative,Sarcastic,Non Vulgar,Abusive,Gender,"মা জানো, পাশের বাড়ির চুলবুলি আমার থেকে বেশি অ...",3,1.0
3,image_ben_5074.jpg,Neutral,Sarcastic,Non Vulgar,Abusive,Gender,Gf কে ফিঙ্গারং করে দেওয়ার পর যখন সে চরমসুখ অনু...,4,1.0
4,image_ben_7351.jpg,Negative,Sarcastic,Vulgar,Abusive,Gender,টিপতে টিপতে যখন ব্যাথ্যা অনুভব হয়,4,1.0


# Prepare Transformer Trainer

In [46]:
class Model(nn.Module):
    def __init__(self, transformer):
        super().__init__()
        self.transformer = transformer 
        
    def m_loss(self, logits, labels):
        return F.binary_cross_entropy_with_logits(logits.squeeze(-1), labels)
        
    def forward(self, **kwargs):
        labels = kwargs.pop("labels", None)
        output = self.transformer(**kwargs)
        output.loss = self.m_loss(output.logits, labels) if labels != None else output.loss
        output.logits = F.sigmoid(output.logits)
        return output
    
    def load(self, model_dir):
        self.transformer.from_pretrained(model_dir)
        print(f"Model loaded: {model_dir}")
    
    def save(self, model_dir):
        self.transformer.save_pretrained(model_dir)
    
class PTTrainer(transformers.Trainer):
    def __init__(self, **kwargs): 
        super().__init__(**kwargs)
        self.compute_metrics = self._metrics if self.eval_dataset is not None else None
        self._patience = 0

    def _metrics(self, eval_preds):
        """
        Evaluation metrics
        """
        threshold = 0.5
        thresholds = np.arange(0.01, 1.0, 0.01) 
        scores, labels = eval_preds
        acc, f1, thr = -1, -1, 0
        auc = sklearn.metrics.roc_auc_score(labels, scores)
        for threshold in thresholds:
            predictions = (scores >= threshold).astype(int)
            new_f1 = sklearn.metrics.f1_score(labels, predictions, average='macro')
            if new_f1 > f1:
                acc = sklearn.metrics.accuracy_score(labels, predictions)
                f1 = new_f1
                thr = threshold
        return {'auc': auc, 'accuracy': acc, 'f1': f1, 'thr': thr}
        
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        forward/backward step
        """
        output = model(**inputs)
        return (output.loss, {'logits': output.logits}) if return_outputs else output.loss

    def _save_checkpoint(self, model, trial, metrics=None):
        # Determine the new best metric / best model checkpoint
        if metrics is not None and self.args.metric_for_best_model is not None:
            metric_to_check = self.args.metric_for_best_model
            if not metric_to_check.startswith("eval_"):
                metric_to_check = f"eval_{metric_to_check}"
            metric_value = metrics[metric_to_check]

            operator = np.greater if self.args.greater_is_better else np.less
            if (
                self.state.best_metric is None
                # or self.state.best_model_checkpoint is None
                or operator(metric_value, self.state.best_metric)
            ):
                self.state.best_metric = metric_value
                # self.state.best_model_checkpoint = self.args.output_dir
                model.save(self.args.output_dir)
                print(f'Model Saved: {metric_value}')
                self._patience = 0
            else:
                print(f"No improvments: {metric_value}, patience: {self._patience}, lr: {self.optimizer.param_groups[0]['lr']}")
                self._patience += 1
            # LR annealing
            if params['learning_rate_scheduling'].lower() == 'cosine_annealing':
                if (self._patience - 1) == int(round(params['patience'] / 2)):
                    if not isinstance(self.lr_scheduler, torch.optim.lr_scheduler.CosineAnnealingLR):
                        self.lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=10, eta_min=max(self.args.learning_rate * 0.01, 1.0e-7), verbose=False)
                        print(f'Starting CosineAnnealingLR schedular.')
                        self._patience -= 1
        else:
            model.save(self.args.output_dir)
            print('Model Saved')


class PTDefaultFlowCallback(transformers.DefaultFlowCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        control = super().on_epoch_end(args, state, control, **kwargs)
        # Evaluate
        if args.evaluation_strategy != transformers.IntervalStrategy.EPOCH \
            and state.epoch == args.num_train_epochs:
            control.should_evaluate = state.best_metric is not None
            control.should_save = True
        return control
    
def get_best_thr(history):
    max_i = -1
    for i, record in enumerate(history):
        if f"eval_{params['metric']}" in record:
            if max_i == -1 or record[f"eval_{params['metric']}"] > history[max_i][f"eval_{params['metric']}"]:
                max_i = i
    return history[max_i]['eval_thr'], history[max_i]['eval_f1']

def apply_thr(y, threshold, mapper=None):
    if mapper is None:
        return (y > threshold).astype(int)
    else:
        return [mapper[x] for x in (y > threshold).astype(int)]

def inference(pred_features, threshold=None):
    print(f"Threshold: {threshold}")
    result = trainer.predict(datasets.Dataset.from_dict(dict(pred_features))).predictions
    if threshold is None:
        test_df[params['label_col']] = result[:, 0]
    else:
        test_df[params['label_col']] = apply_thr(result[:, 0], threshold, y_map)
    return test_df[[col for col in test_df.columns if col != 'text']]

In [47]:
transformer = transformers.AutoModelForSequenceClassification.from_pretrained(params['model_id'], num_labels=1)
_tokenizer = transformers.AutoTokenizer.from_pretrained(params['model_id'], model_max_length=params['max_seq'])
callbacks = [PTDefaultFlowCallback(), 
             transformers.EarlyStoppingCallback(early_stopping_patience=params['patience'])]

if isinstance(params['logging_steps'], int):
    save_strategy = transformers.IntervalStrategy.STEPS
else:
    save_strategy = transformers.IntervalStrategy.EPOCH 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at models--l3cube-pune--bengali-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
def get_trainer(valid_split=0):
    _model = Model(copy.deepcopy(transformer))
    optimizer = torch.optim.AdamW(_model.parameters(), lr=params['learning_rate'])
    model_dir = f"{params['model_dir']}{valid_split}"
    t_df, v_df = train_df[train_df['split'] != valid_split], train_df[train_df['split'] == valid_split]
    print(f"Train: {len(t_df)}, Valid: {len(v_df)}")
    train_features = prepare_feature(t_df, params['label_col'], _tokenizer, max_length=params['train_max_length'])
    valid_features = prepare_feature(v_df, params['label_col'], _tokenizer, max_length=params['train_max_length'])
    trainer = PTTrainer(model=_model,
        train_dataset=datasets.Dataset.from_dict(dict(train_features)),
        eval_dataset=datasets.Dataset.from_dict(dict(valid_features)) if valid_features is not None else None,
        args=transformers.TrainingArguments(
            label_names=['labels'],
            seed=params['seed'],
            data_seed=params['data_seed'],
            remove_unused_columns=False,
            report_to="tensorboard" if params['tensorboard_dir'] is not None else "none",
            logging_dir=params['tensorboard_dir'],
            per_device_train_batch_size=params['train_batch_size'], 
            per_device_eval_batch_size=params['eval_batch_size'],
            gradient_accumulation_steps=params['gradient_accumulation_steps'],
            warmup_steps=params['warmup_proportion'], 
            # max_steps=20,
            num_train_epochs=params['num_train_epochs'],
            learning_rate=params['learning_rate'], 
            fp16=params['fp16'],
            metric_for_best_model=params['metric'],
            # gradient_checkpointing=fp16,
            # evaluate_during_training=True,
            logging_steps=params['logging_steps'], 
            output_dir=model_dir,
            evaluation_strategy=save_strategy,
            save_strategy=save_strategy,
            logging_strategy=save_strategy,
            save_total_limit=1,
            greater_is_better=params['metric_mode'], 
            load_best_model_at_end=True,
            eval_steps=params['logging_steps'],
            save_steps=params['logging_steps'],
            overwrite_output_dir=True,
            max_grad_norm=1.0,
        ),
        optimizers=[optimizer, transformers.get_constant_schedule(optimizer)],
        callbacks=callbacks,
        data_collator=transformers.DataCollatorWithPadding(_tokenizer),
    )
    return _model, trainer, model_dir

# Train / Inference

In [49]:
pred_features = prepare_feature(test_df, params['label_col'], _tokenizer, max_length=params['eval_max_length'])
subm_df = None
thresholds = []
scores = []
cols = []
for fold in range(params['n_splits']):
    print(f"Fold: {fold + 1} out of {params['n_splits']}")
    col = f'y_{fold}'
    model, trainer, model_dir = get_trainer(fold)
    trainer.train()
    trainer.model.load(model_dir)
    subm = inference(pred_features)
    t, f1 = get_best_thr(trainer.state.log_history)
    thresholds.append(t)
    scores.append(f1)
    if subm_df is None:
        subm_df = subm
    subm_df[col] = subm[params['label_col']]
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    cols.append(col)

Fold: 1 out of 10
Train: 2423, Valid: 270


D:\MiniConda\envs\pythonProject18\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Auc,Accuracy,F1,Thr
1,0.670800,0.642109,0.569981,0.725926,0.433417,0.440000
2,0.628600,0.612781,0.541264,0.725926,0.420601,0.390000
3,0.606000,0.596858,0.550124,0.725926,0.420601,0.340000
4,0.593800,0.589802,0.569843,0.725926,0.420601,0.310000
5,0.589900,0.588186,0.555261,0.725926,0.420601,0.300000
6,0.589000,0.587786,0.532543,0.725926,0.420601,0.290000


Model Saved: 0.4334165154264973
No improvments: 0.4206008583690987, patience: 0, lr: 1e-05
No improvments: 0.4206008583690987, patience: 1, lr: 1e-05
No improvments: 0.4206008583690987, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.4206008583690987, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.4206008583690987, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/0
Threshold: None


Fold: 2 out of 10
Train: 2423, Valid: 270


...

Model Saved: 0.4202586206896552
No improvments: 0.4202586206896552, patience: 0, lr: 1e-05
No improvments: 0.4202586206896552, patience: 1, lr: 1e-05
No improvments: 0.4202586206896552, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.4202586206896552, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.4202586206896552, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/8
Threshold: None


Fold: 10 out of 10
Train: 2424, Valid: 269


D:\MiniConda\envs\pythonProject18\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Auc,Accuracy,F1,Thr
1,0.670500,0.640591,0.452824,0.728625,0.421505,0.440000
2,0.628000,0.612093,0.521002,0.728625,0.421505,0.390000
3,0.606200,0.595380,0.502027,0.728625,0.421505,0.340000
4,0.594500,0.587736,0.526454,0.728625,0.421505,0.310000
5,0.590200,0.586191,0.519290,0.728625,0.421505,0.300000
6,0.589300,0.585363,0.549273,0.728625,0.421505,0.290000


Model Saved: 0.421505376344086
No improvments: 0.421505376344086, patience: 0, lr: 1e-05
No improvments: 0.421505376344086, patience: 1, lr: 1e-05
No improvments: 0.421505376344086, patience: 2, lr: 1e-05
Starting CosineAnnealingLR schedular.
No improvments: 0.421505376344086, patience: 2, lr: 6.579634122155956e-06
No improvments: 0.421505376344086, patience: 3, lr: 1.0453658778440268e-06
Model loaded: /model/9
Threshold: None


In [50]:
thresholds, scores

([0.44, 0.44, 0.44, 0.44, 0.44, 0.23, 0.44, 0.32, 0.44, 0.44],
 [0.4334165154264973,
  0.4206008583690987,
  0.4305694305694306,
  0.4202586206896552,
  0.4202586206896552,
  0.7195845416121757,
  0.4202586206896552,
  0.6774150566289141,
  0.4202586206896552,
  0.421505376344086])

In [54]:
thr = np.array(thresholds).mean()
thr = 0.38
print(f"Best threshold: {thr}")
print(f"Best score: {np.array(scores).mean()}")
subm_df['avg'] = subm_df[cols].apply('mean', axis=1)
subm_df[params['label_col']] = apply_thr(subm_df['avg'], thr, y_map)

Best threshold: 0.38
Best score: 0.4784126261708823


In [55]:
subm_df.head(40)

,Ids,OCR,task_4,y_0,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,avg
0,image_ben_1422.png,বৌ-এর প্রেগনেন্সি রিপোর্ট পজিটিভ জানার পর বর চ...,Abusive,0.288952,0.288376,0.288850,0.286830,0.288644,0.806509,0.287427,0.716673,0.287545,0.288914,0.382872
1,image_ben_3908.jpg,কিন্তু মিষ্টতা নেই,Non-abusive,0.289022,0.288287,0.288786,0.287114,0.289142,0.113413,0.287651,0.177219,0.287506,0.288677,0.259682
2,image_ben_4634.jpg,মানুষ যা দেখে প্রথম সিঙারা আবিষ্কারের ধারণা পায়,Non-abusive,0.288665,0.288058,0.288789,0.286801,0.288819,0.112949,0.287380,0.177064,0.287397,0.288756,0.259468
3,image_ben_3192.png,এক মেয়ে ফ্যানে ওড়না পাচিয়ে সুই*সাইড করতে যাচ্ছ...,Non-abusive,0.289117,0.288216,0.288736,0.286846,0.288667,0.806471,0.287471,0.189062,0.287493,0.289032,0.330111
4,image_ben_4836.jpg,No text detected,Non-abusive,0.288921,0.288657,0.288618,0.286954,0.288810,0.140698,0.287619,0.177425,0.287726,0.288943,0.262437
5,image_ben_3907.jpg,বাঘের চামড়া দিয়ে বিড়াল ঢেকে রাখার একটি চিত্র।,Non-abusive,0.288799,0.288116,0.288844,0.286988,0.288819,0.409049,0.287567,0.527286,0.287481,0.288838,0.324179
6,image_ben_4636.jpg,"আমার উপর রেগে আছো রিয়া? না, রেগে নেই তাহলে মুখ...",Non-abusive,0.288830,0.287960,0.288745,0.286842,0.288714,0.402686,0.287533,0.716284,0.287453,0.288860,0.342391
7,image_ben_5080.jpg,নিঃস্ব উপেন তার দুই বিঘা জমি,Non-abusive,0.288723,0.288281,0.288668,0.286887,0.288898,0.113022,0.287421,0.177105,0.287541,0.288862,0.259541
8,image_ben_3704.jpg,"চলো চিন্টু মজা হবে ! Yes, chumki তাড়াতাড়ি এসো ...",Abusive,0.288897,0.288073,0.288664,0.286803,0.288729,0.805769,0.287389,0.716594,0.287463,0.288905,0.382729
9,image_ben_5071.jpg,No text detected,Non-abusive,0.288921,0.288657,0.288618,0.286954,0.288810,0.140698,0.287619,0.177425,0.287726,0.288943,0.262437


In [56]:
subm_df[['Ids', params['label_col']]].to_csv('submission_task4.csv', index=False)